In [11]:
import sqlite3
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('default')

### 1. Import the db file

In [12]:
db = sqlite3.connect("baseline_v1.4_10yrs.db")

### 2. Define the "cursor"

In [13]:
cursor = db.cursor()

### 3. See different tables 

In [14]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('SummaryAllProps',), ('info',), ('Proposal',)]


### 4. Get info about the tables with in the table

In [15]:
cursor.execute("PRAGMA table_info(SummaryAllProps)")
print(cursor.fetchall())

[(0, 'observationId', 'INTEGER', 0, None, 0), (1, 'fieldRA', 'REAL', 0, None, 0), (2, 'fieldDec', 'REAL', 0, None, 0), (3, 'observationStartMJD', 'REAL', 0, None, 0), (4, 'flush_by_mjd', 'REAL', 0, None, 0), (5, 'visitExposureTime', 'REAL', 0, None, 0), (6, 'filter', 'TEXT', 0, None, 0), (7, 'rotSkyPos', 'REAL', 0, None, 0), (8, 'numExposures', 'INTEGER', 0, None, 0), (9, 'airmass', 'REAL', 0, None, 0), (10, 'seeingFwhm500', 'REAL', 0, None, 0), (11, 'seeingFwhmEff', 'REAL', 0, None, 0), (12, 'seeingFwhmGeom', 'REAL', 0, None, 0), (13, 'skyBrightness', 'REAL', 0, None, 0), (14, 'night', 'INTEGER', 0, None, 0), (15, 'slewTime', 'REAL', 0, None, 0), (16, 'visitTime', 'REAL', 0, None, 0), (17, 'slewDistance', 'REAL', 0, None, 0), (18, 'fiveSigmaDepth', 'REAL', 0, None, 0), (19, 'altitude', 'REAL', 0, None, 0), (20, 'azimuth', 'REAL', 0, None, 0), (21, 'paraAngle', 'REAL', 0, None, 0), (22, 'cloud', 'REAL', 0, None, 0), (23, 'moonAlt', 'REAL', 0, None, 0), (24, 'sunAlt', 'REAL', 0, None, 0

### 5. Print length of the entire data set

In [16]:
cursor.execute("SELECT fieldid, observationStartLST, filter, fiveSigmaDepth FROM SummaryAllProps") 
summaryCursor = np.array(cursor.fetchall()) #NOTE: this takes a LONG time
print(len(summaryCursor[:,1].astype('float')))

2180589


### 6. Print out all of the different "tables" 

In [17]:
cursor.execute("PRAGMA table_info(SummaryAllProps)")
print(cursor.fetchall()) 

[(0, 'observationId', 'INTEGER', 0, None, 0), (1, 'fieldRA', 'REAL', 0, None, 0), (2, 'fieldDec', 'REAL', 0, None, 0), (3, 'observationStartMJD', 'REAL', 0, None, 0), (4, 'flush_by_mjd', 'REAL', 0, None, 0), (5, 'visitExposureTime', 'REAL', 0, None, 0), (6, 'filter', 'TEXT', 0, None, 0), (7, 'rotSkyPos', 'REAL', 0, None, 0), (8, 'numExposures', 'INTEGER', 0, None, 0), (9, 'airmass', 'REAL', 0, None, 0), (10, 'seeingFwhm500', 'REAL', 0, None, 0), (11, 'seeingFwhmEff', 'REAL', 0, None, 0), (12, 'seeingFwhmGeom', 'REAL', 0, None, 0), (13, 'skyBrightness', 'REAL', 0, None, 0), (14, 'night', 'INTEGER', 0, None, 0), (15, 'slewTime', 'REAL', 0, None, 0), (16, 'visitTime', 'REAL', 0, None, 0), (17, 'slewDistance', 'REAL', 0, None, 0), (18, 'fiveSigmaDepth', 'REAL', 0, None, 0), (19, 'altitude', 'REAL', 0, None, 0), (20, 'azimuth', 'REAL', 0, None, 0), (21, 'paraAngle', 'REAL', 0, None, 0), (22, 'cloud', 'REAL', 0, None, 0), (23, 'moonAlt', 'REAL', 0, None, 0), (24, 'sunAlt', 'REAL', 0, None, 0

### 7. Select desired values to put into a data frame

In [18]:
MJD= cursor.execute("select observationStartMJD FROM SummaryAllProps")
MJD= cursor.fetchall() 
MJD=[i[0] for i in MJD] 
# Start Date for LSST is 59853.985644
Days_Since_Start = MJD - np.array(59853.985644) 


RA = cursor.execute("select fieldra FROM SummaryAllProps") 
RA=[i[0] for i in RA]

Dec = cursor.execute("select fieldDec FROM SummaryAllProps") 
Dec = cursor.fetchall()
Dec=[i[0] for i in Dec]

skybrightness = cursor.execute("select skyBrightness FROM SummaryAllProps")
skybrightness = cursor.fetchall()
skybrightness = [i[0] for i in skybrightness]

moonphase = cursor.execute("select moonphase FROM SummaryAllProps")
moonphase = cursor.fetchall()
moonphase = [i[0] for i in moonphase]

cloud = cursor.execute("select cloud FROM SummaryAllProps")
cloud = cursor.fetchall()
cloud = [i[0] for i in cloud]

airmass = cursor.execute("select airmass FROM SummaryAllProps")
airmass = cursor.fetchall()
airmass = [i[0] for i in airmass]

filter = cursor.execute("select filter FROM SummaryAllProps")
filter = cursor.fetchall()
filter =[i[0] for i in filter]

#Convert Filter g,b,v,r,i into numbers that can be shown on the colormap
# filter_num = []
# for i in range(0,len(filter)): 
#     if filter[i] == 'g':
#         filter[i] = 1e4
#     if filter[i] == 'b':
#         filter[i] == 1
    
#     filter_num.append(filter)

### 8. Put values into data  frame

In [21]:
obsv = {
        'Days_Since_Start': Days_Since_Start,
        'RA': RA,
        'Dec': Dec,
        'skybrightness': skybrightness, 
        'moonphase': moonphase,
        'cloud': cloud,
        'airmass': airmass,
        'filter':filter
        }

df = pd.DataFrame(obsv, columns= [ 'Days_Since_Start','RA','Dec','skybrightness','moonphase','cloud','airmass','filter'])
df = (df[0:2180589:50]) #There are 209799 values for JUST the green filter. Count by 100+ to reduce length 

df


,Days_Since_Start,RA,Dec,skybrightness,moonphase,cloud,airmass,filter
0,-1.791486e-07,294.696105,4.470982,18.304678,42.296595,0.0,1.216566,g
50,2.207091e-02,315.147446,-50.445865,18.530188,42.403654,0.0,1.080051,g
100,4.286855e-02,310.750035,-68.862225,18.364862,42.509076,0.0,1.279886,g
150,6.384219e-02,321.326185,-44.140467,18.577686,42.621052,0.0,1.031300,g
200,8.485535e-02,320.993922,-62.831260,18.439284,42.739344,0.0,1.188577,g
...,...,...,...,...,...,...,...,...
2180350,3.652300e+03,64.503542,-33.170697,22.254113,19.718625,0.0,1.055799,z
2180400,3.652322e+03,54.605726,-12.230438,22.154158,19.569410,0.0,1.053610,z
2180450,3.652346e+03,41.874745,-15.042425,21.095537,19.419849,0.0,1.079142,z
2180500,3.652370e+03,48.871737,-7.585806,20.288312,19.274071,0.0,1.142958,z


### 9. Export data frame to CSV file

In [22]:
df.to_csv(r'/Users/candicestauffer/desktop/VIS_proj/src/data/Rubin_data.csv', index = True)